In [12]:
# Cargamos algunas librerías necesarias.
import numpy  as np  
import pandas as pd
pd.set_option('display.max_columns', 100) # Con esto garantizo el ver todas las columnas.

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt # Para dibujar alguna gráfica.
%matplotlib inline

from geopy.geocoders import Nominatim
import geocoder
import requests
import re
import threading 
import time
from math import sin, cos, sqrt, atan2, radians
import googlemaps
from geopy.geocoders import Nominatim

In [2]:
df = pd.read_csv('./Accidentes_y_ocio.csv',sep=';', decimal='.', encoding= 'utf-8')
df.head()

,Unnamed: 0,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad,Latitude,Longitude,Distancia ocio Km,Lejania ocio
0,0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 21 A 24 AÑOS,40.483873,-3.673592,1.508801,muy lejos
1,1,04/03/2014,DE 13:00 A 13:59,MARTES,VICALVARO,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,0,2014/2147,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,COLISIÓN DOBLE,TURISMO,VIAJERO,MUJER,HL,DE 18 A 20 AÑOS,40.398454,-3.601018,2.790873,muy lejos
2,2,26/05/2018,DE 00:00 A 00:59,SABADO,ARGANZUELA,CALLE DE EROS - CALLE DE LA ESTRELLA HADAR,0,2018/5408,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 35 A 39 AÑOS,40.393845,-3.680282,0.948289,lejos
3,3,25/02/2010,DE 14:00 A 14:59,JUEVES,CARABANCHEL,CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO,0,2010/1766,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 30 A 34 ANOS,40.377139,-3.739303,0.984928,lejos
4,4,17/07/2012,DE 18:00 A 18:59,MARTES,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA - AUTOVIA M-40,0,2012/6707,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,3.0,COLISIÓN DOBLE,FURGONETA,CONDUCTOR,HOMBRE,IL,DE 21 A 24 AÑOS,40.410009,-3.641304,1.041850,muy lejos


In [3]:
df_radares = pd.read_csv('./radares.csv',sep=';', decimal='.', encoding= 'utf-8')
df_radares.head()

,Unnamed: 0,Ubicacion,Latitude,Longitude
0,0,"M-30, ENTRADA DE LA A-1, SENTIDO PUENTE DE VEN...",40.479341,-3.674338
1,1,"M-30, ENTRADA DE LA A-1, SENTIDO PUENTE DE VEN...",40.479342,-3.674291
2,2,"PASO INFERIOR COSTA RICA, SENTIDO PRÍNCIPE DE ...",40.458923,-3.668803
3,3,"M-30, PTE. DE VENTAS. SENTIDO SUR A O´DONNELL ...",40.426180,-3.660144
4,4,"M-30, PTE. DE VENTAS. SENTIDO SUR A O´DONNELL ...",40.426190,-3.660061


In [4]:
def distancia(long1,lat1,long2,lat2):
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(long1)
    lat2 = radians(lat2)
    lon2 = radians(long2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

In [10]:
def comparador(row):
    lat = row['Latitude']
    long = row['Longitude']
    v = list()
    for i in range(len(df_radares)):
        c = distancia(df_radares['Longitude'][i],df_radares['Latitude'][i], long, lat )
        v.append(c)
    return min(v)
    

In [13]:
df['Distancia_radares'] = df[['Latitude','Longitude']].apply(lambda row: comparador(row), axis=1)

In [14]:
df.head()

,Unnamed: 0,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad,Latitude,Longitude,Distancia ocio Km,Lejania ocio,Distancia_radares
0,0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 21 A 24 AÑOS,40.483873,-3.673592,1.508801,muy lejos,0.507497
1,1,04/03/2014,DE 13:00 A 13:59,MARTES,VICALVARO,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,0,2014/2147,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,COLISIÓN DOBLE,TURISMO,VIAJERO,MUJER,HL,DE 18 A 20 AÑOS,40.398454,-3.601018,2.790873,muy lejos,5.331349
2,2,26/05/2018,DE 00:00 A 00:59,SABADO,ARGANZUELA,CALLE DE EROS - CALLE DE LA ESTRELLA HADAR,0,2018/5408,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 35 A 39 AÑOS,40.393845,-3.680282,0.948289,lejos,0.950698
3,3,25/02/2010,DE 14:00 A 14:59,JUEVES,CARABANCHEL,CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO,0,2010/1766,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 30 A 34 ANOS,40.377139,-3.739303,0.984928,lejos,2.881995
4,4,17/07/2012,DE 18:00 A 18:59,MARTES,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA - AUTOVIA M-40,0,2012/6707,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,3.0,COLISIÓN DOBLE,FURGONETA,CONDUCTOR,HOMBRE,IL,DE 21 A 24 AÑOS,40.410009,-3.641304,1.041850,muy lejos,1.689948


In [15]:
df['Lejania_radares']=''
for x in range(len(df)):
    if df['Distancia_radares'][x]<0.2:
        df['Lejania_radares'][x]='Muy cerca'
    elif df['Distancia_radares'][x]>=0.2 and df['Distancia_radares'][x]<0.5:
        df['Lejania_radares'][x]='Cerca'
    elif df['Distancia_radares'][x]>=0.5 and df['Distancia_radares'][x]<1:
        df['Lejania_radares'][x]='lejos'
    else:
        df['Lejania_radares'][x]='muy lejos'

C:\Users\migue\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\migue\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\migue\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Us

In [16]:
df.head(10)

,Unnamed: 0,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad,Latitude,Longitude,Distancia ocio Km,Lejania ocio,Distancia_radares,Lejania_radares
0,0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 21 A 24 AÑOS,40.483873,-3.673592,1.508801,muy lejos,0.507497,lejos
1,1,04/03/2014,DE 13:00 A 13:59,MARTES,VICALVARO,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,0,2014/2147,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,COLISIÓN DOBLE,TURISMO,VIAJERO,MUJER,HL,DE 18 A 20 AÑOS,40.398454,-3.601018,2.790873,muy lejos,5.331349,muy lejos
2,2,26/05/2018,DE 00:00 A 00:59,SABADO,ARGANZUELA,CALLE DE EROS - CALLE DE LA ESTRELLA HADAR,0,2018/5408,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 35 A 39 AÑOS,40.393845,-3.680282,0.948289,lejos,0.950698,lejos
3,3,25/02/2010,DE 14:00 A 14:59,JUEVES,CARABANCHEL,CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO,0,2010/1766,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 30 A 34 ANOS,40.377139,-3.739303,0.984928,lejos,2.881995,muy lejos
4,4,17/07/2012,DE 18:00 A 18:59,MARTES,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA - AUTOVIA M-40,0,2012/6707,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,3.0,COLISIÓN DOBLE,FURGONETA,CONDUCTOR,HOMBRE,IL,DE 21 A 24 AÑOS,40.410009,-3.641304,1.041850,muy lejos,1.689948,muy lejos
5,5,31/05/2016,DE 13:00 A 13:59,MARTES,TETUAN,AVENIDA DEL GENERAL PERON - CALLE DEL CAPITAN ...,0,2016/5276,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HG,DE 35 A 39 AÑOS,40.452295,-3.692716,0.100733,Muy cerca,2.154023,muy lejos
6,6,16/08/2011,DE 12:00 A 12:59,MARTES,CENTRO,CALLE DEL ARENAL,23,2011/7464,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,ATROPELLO,TURISMO,CONDUCTOR,HOMBRE,IL,DE MAS DE 74 AÑOS,40.429208,-3.784634,3.896086,muy lejos,3.693610,muy lejos
7,7,19/01/2012,DE 20:00 A 20:59,JUEVES,TETUAN,PASEO DE LA DIRECCION - CALLE DE VILLAAMIL,0,2012/625,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 25 A 29 AÑOS,40.460472,-3.707914,1.222447,muy lejos,2.379076,muy lejos
8,8,02/12/2015,DE 6:00 A 6:59,MIERCOLES,TETUAN,CALLE DE LAS MAGNOLIAS - CALLE DE SAN LEOPOLDO,0,2015/11153,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,COLISIÓN DOBLE,TURISMO,CONDUCTOR,HOMBRE,IL,DE 55 A 59 AÑOS,40.469716,-3.691720,0.625634,lejos,1.310666,muy lejos
9,9,15/10/2016,DE 18:00 A 18:59,SABADO,ARGANZUELA,AUTOVIA M-30 CALZADA 1 KM.,15000,2016/9662,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,3.0,CHOQUE CON OBJETO FIJO,TURISMO,VIAJERO,HOMBRE,HL,DE 25 A 29 AÑOS,40.396602,-3.703340,0.140841,Muy cerca,0.186728,Muy cerca


In [17]:
df['Lejania_radares'].value_counts(dropna=False)

muy lejos    155696
lejos         29234
Cerca         11033
Muy cerca      4058
Name: Lejania_radares, dtype: int64

In [18]:
df.to_csv('Ocio_radares.csv',sep=';', decimal='.', index=False)